In [ ]:
#Primero hay que llamar a unas librerías que el programa necesitará.
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

In [ ]:
#Cargamos el fichero de datos. Formato en Excel, csv.
AH_data = pd.read_csv("dia1.csv")

In [ ]:
AH_data.drop('Tipo de Suelo', axis=1, inplace=True)
AH_data.drop('Análisis', axis=1, inplace=True)

In [ ]:
AH_data.head()

In [ ]:
#Eliminamos los datos con valores missing porque Python no puede hacer árboles con datos missing
data_clean = AH_data.dropna()

In [ ]:
#Para comprobar que se ha leído bien, podemos lista las variables en el fichero y sacar los principales estadísticos
data_clean.dtypes

In [ ]:
#Principales estadísticos
data_clean.describe()

In [ ]:
data_clean.keys()

In [ ]:
#Indicamos las variables predictoras y debajo la variable objetivo. Cada uno con los nombres de las variables que tenéis en el fichero csv.
predictors = data_clean[['T° Ambiental (°C) BRUTO', 'Velocidad del viento (KM/H)', 'Valor de tipo de suelo','Humedad Suelo (%)',  'T° Suelo (°C)']] 

In [ ]:
targets = data_clean['Análisis valor']

In [ ]:
#Creamos la muestra de entrenamiento y de test, tanto para predictores como para la variable objetivo, siendo test el 40%
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=.4)

In [ ]:
#Comprobamos el tamaño de las diferentes muestras (pred=predictora; tar=target, objetivo). La salida en cada caso es una pareja de datos: el tamaño de la muestra y el número de variables
print(pred_train.shape,
pred_test.shape,
tar_train.shape,
tar_test.shape)

In [ ]:
#Construimos el árbol con los datos de entrenamiento
classifier=DecisionTreeClassifier()
classifier=classifier.fit(pred_train,tar_train)

In [ ]:
#Predecimos para los valores del grupo Test
predictions=classifier.predict(pred_test)

In [ ]:
#Pedimos la matriz de confusión de las predicciones del grupo Test. La diagonal de esta matriz se lee: arriba a la izda True Negatives y abajo a la dcha True Positives. 
sklearn.metrics.confusion_matrix(tar_test,predictions)

In [ ]:
#Sacamos el índice Accuracy Score, que resume la Matriz de Confusión y la cantidad de aciertos.
sklearn.metrics.accuracy_score(tar_test, predictions)

In [ ]:
#Para dibujar el árbol hay que importar otra serie de cosas
from sklearn import tree
from io import StringIO
from IPython.display import Image

In [ ]:
resultado=['regar','no regar']

In [ ]:
#Pintamos el árbol
out = StringIO()
tree.export_graphviz(classifier, out_file='treeMacarena.dot', feature_names=data_clean.keys(),filled=True,rounded=True
                ,proportion=True,class_names=resultado)


In [ ]:
from PIL import Image

In [ ]:
i = Image.open('tree.png','r') 

In [ ]:
i

In [ ]:
#comando
#dot -Tpng treeMacarena.dot -o tree_simple.png

In [ ]:
#j = Image.open('tree_simple.png','r') 
#j